In [1]:
import requests

use_real_puzzle_data = False # else use demo data
separator = '\n' if True else ','

github_site_user = "https://raw.githubusercontent.com/stephenhumphrey/"
repository_branch = "advent-of-code/main/2021/12/18/"

names_files = [("magnitudes","demo-magnitudes"),
               ("demo","demo-summations"),
               ("puzzle","puzzle-data")]

data = {source:{"name":source,
                "url":f"{github_site_user}{repository_branch}{file}.txt"} 
        for source,file in names_files}

for name,source in data.items():
    print(f"{name}: {source['url']}")

magnitudes: https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/18/demo-magnitudes.txt
demo: https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/18/demo-summations.txt
puzzle: https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/18/puzzle-data.txt


In [2]:
# cheap and dirty constants
infinity = 99999999999999999999999999

# allow for easily turning on and off local logging inside this notebook
do_verbose = False # set to true for copious trace logging
def throwaway_verbose(unused):
    return
verbose = print if do_verbose else throwaway_verbose

# for sanity, let's limit Jupyter from getting a firehose of logging,
# which it handles poorly
global_endless_loop = 299 # halts certain functions which might fail

In [3]:
# utility function for pretty-printing large data arrays
def verbose_list(plist_string,max_elements = 5):
    plist = eval(plist_string)
    if len(plist) <= max_elements:
        sub = f"{plist}"
    elif max_elements < 2:
        sub = f"[{plist[0]}, ...]"
    else:
        half_max = max_elements // 2
        sub = "["
        for v in plist[0:half_max]:
            sub += f"{v}, "
        sub += "... "
        for v in plist[-half_max:-1]:
            sub += f"{v}, "
        sub += f"{plist[-1]}]"
        
    print(f"len({plist_string})={len(plist)}, {plist_string}={sub}")

In [4]:
# download and roughly process our testing and production data sources
for source_name,source in data.items():
    raw_data = requests.get(source["url"]).text
    data_lines = [line for line in raw_data.split(separator) if len(line.strip()) > 0]
    source["raw_lines"] = data_lines
    
    if False:
        print(source_name)
        for line in data_lines:
            print(line)
    
    if source_name == "puzzle":
        source["operands_raw"] = [data_lines]
        source["expected_values"] = ["unknown"]
    else:
        right = []
        source["operands_raw"] = []
        for line in data_lines:
            line_data = [v.strip() for v in line.split('=') if len(v.strip())>1]
            left = [v.strip() for v in line_data[0].split('+')]
            source["operands_raw"].append(left)
            right.append(line_data[1])
        source["expected_values"] = right

In [5]:
# utility functions for dealing with this particular puzzle's data types
def separate_pair(pair_string):
    assert(pair_string[0]=='[' and pair_string[-1]==']')
    
    depth = 0
    split_at = 0
    for i,c in enumerate(pair_string):
        if c == '[':
            depth += 1
        elif c == ']':
            depth -= 1
        elif c == ',' and depth == 1:
            split_at = i
            break
    return pair_string[1:split_at], pair_string[split_at + 1:-1]

def parse_pair(pair_string):
    
    if pair_string[0] != '[':
        return int(pair_string)
    
    assert(pair_string[0]=='[' and pair_string[-1]==']')

    left,right = separate_pair(pair_string)
    left_parsed = parse_pair(left)
    right_parsed = parse_pair(right)
    
    return [left_parsed,right_parsed]

def pair_magnitude(pair):
    assert(isinstance(pair,list) and len(pair)==2)
    
    left,right = pair[0], pair[1]
    assert(isinstance(left,int) or isinstance(left,list))
    assert(isinstance(right,int) or isinstance(right,list))
    
    sum_left_right = 0
    if isinstance(left,int):
        sum_left_right += left * 3
    else:
        sum_left_right += pair_magnitude(left) * 3
    
    if isinstance(right,int):
        sum_left_right += right * 2
    else:
        sum_left_right += pair_magnitude(right) * 2

    return sum_left_right

In [6]:
# process and normalize our testing and production datasets so they share formats
for source_name in ["demo"]: # in data.keys():
    source = data[source_name]
    expected_values = source["expected_values"]
    pairs = []
    source["operands"] = pairs
    for i,operands in enumerate(source["operands_raw"]):
        
        verbose(f"{source_name}{i} operands({len(operands)}) expect: {expected_values[i]}")
        
        operand_pairs = []
        for j,operand in enumerate(operands):
            pair = parse_pair( operand )
            operand_pairs.append(pair)
            magnitude = pair_magnitude( pair )
            
            verbose(f"  operand[{j}] = {operand} pair = {pair} magnitude = {magnitude}")
        
        verbose(f"  pairs = {operand_pairs}")
        pairs.append(operand_pairs)

In [7]:
# this puzzle's pair summation, not counting "reduction" (as the puzzle defines it)
def unreduced_sum_of_two_pairs(pair0, pair1):
    if pair0 is None:
        return pair1
    if pair1 is None:
        return pair0
    return [pair0, pair1]

In [8]:
notes = """
To reduce a snailfish number, you must repeatedly do the first action in this list that applies to the snailfish number:

If any pair is nested inside four pairs, the leftmost such pair explodes.
If any regular number is 10 or greater, the leftmost such regular number splits.
Once no action in the above list applies, the snailfish number is reduced.

During reduction, at most one action applies, after which the process returns to the top of the list of actions. For example, if split produces a pair that meets the explode criteria, that pair explodes before other splits occur.

To explode a pair, the pair's left value is added to the first regular number to the left of the exploding pair (if any), and the pair's right value is added to the first regular number to the right of the exploding pair (if any). Exploding pairs will always consist of two regular numbers. Then, the entire exploding pair is replaced with the regular number 0.
Here are some examples of a single explode action:

[[[[[9,8],1],2],3],4] becomes [[[[0,9],2],3],4] (the 9 has no regular number to its left, so it is not added to any regular number).
[7,[6,[5,[4,[3,2]]]]] becomes [7,[6,[5,[7,0]]]] (the 2 has no regular number to its right, and so it is not added to any regular number).
[[6,[5,[4,[3,2]]]],1] becomes [[6,[5,[7,0]]],3].
[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]] becomes [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]] (the pair [3,2] is unaffected because the pair [7,3] is further to the left; [3,2] would explode on the next action).
[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]] becomes [[3,[2,[8,0]]],[9,[5,[7,0]]]].
To split a regular number, replace it with a pair; the left element of the pair should be the regular number divided by two and rounded down, while the right element of the pair should be the regular number divided by two and rounded up. For example, 10 becomes [5,5], 11 becomes [5,6], 12 becomes [6,6], and so on.

Here is the process of finding the reduced result of [[[[4,3],4],4],[7,[[8,4],9]]] + [1,1]:

after addition: [[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]
after explode:  [[[[0,7],4],[7,[[8,4],9]]],[1,1]]
after explode:  [[[[0,7],4],[15,[0,13]]],[1,1]]
after split:    [[[[0,7],4],[[7,8],[0,13]]],[1,1]]
after split:    [[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]
after explode:  [[[[0,7],4],[[7,8],[6,0]]],[8,1]]
Once no reduce actions apply, the snailfish number that remains is the actual result of the addition operation: [[[[0,7],4],[[7,8],[6,0]]],[8,1]].
"""

In [9]:
# process summation of multiple pairs in an equation, sequentionally
def reduced_sum_of_pairs(pairs):
    assert(len(pairs) > 0)
    
    print(f"  called reduced_sum_of_pairs() with {pairs}")
    
    previous_sum = None
    for pair in pairs:
        unreduced_sum = unreduced_sum_of_two_pairs(previous_sum,pair)
        print(f"  *** unreduced_sum = {unreduced_sum}")
        while reduce( unreduced_sum ):
            pass
        previous_sum = unreduced_sum        
    
    return previous_sum

In [10]:
# search the tree for a particular pair, finding the closest "regular" neighbor as well
def find_side_of(exploding_pair,parents,use_strongside = True):
    
    verbose(f"    calling find_side_of({'strong' if use_strongside else 'weak'}) "
          f"pair = {exploding_pair} parents = {parents}")
    
    found = False
    neighbor = None
    closer_neighbor = None
    immediate_parent = None
    
    if isinstance(parents,int): 
        # this level isn't a pair
        return found, neighbor, immediate_parent 
    
    strong_indices = (0,1) if use_strongside else (1,0)
    strong_index, weak_index = strong_indices
    
    strongside, weakside = parents[strong_index], parents[weak_index]
    
    if exploding_pair is strongside:
        # we are done searching; there can't be any closer strongside neighbor
        found = True
        immediate_parent = parents
        
    elif exploding_pair is weakside:
        # we found the pair, but there might still be a closer strongside neighbor
        found = True
        immediate_parent = parents
        
        # search only for a closer neighbor in the strongside tree
        if isinstance(strongside,int):
            neighbor = (strongside, parents, use_strongside)
        else:
            expectFalse, closer_neighbor, _ = find_side_of( exploding_pair,
                                                 strongside, use_strongside )
            assert(not expectFalse) # we better not find it again
        
    else:
        # we still haven't found the pair, it could be on either side
        # search the strongside tree first, and failing that, the weakside tree
        if isinstance(strongside,int):
            neighbor = (strongside, parents, use_strongside)
        else:
            found, closer_neighbor, immediate_parent = find_side_of( exploding_pair,
                                                        strongside, use_strongside )
        if closer_neighbor is not None:
            neighbor = closer_neighbor
            closer_neighbor = None
        
        if not found: 
            # the exploding pair wasn't anywhere in the strongside tree
            # search the weakside
            if isinstance(weakside,int):
                neighbor = (weakside, parents, not use_strongside)
            else:
                found, closer_neighbor, immediate_parent = find_side_of( exploding_pair,
                                                        weakside, use_strongside )
    
    # final cleanup, we may have found a closer neighbor than our prior closest
    if closer_neighbor is not None:
        neighbor = closer_neighbor
    
    verbose(f"    exiting find_side_of({'strong' if use_strongside else 'weak'}) "
          f"found = {'found' if found else 'not-found'}"
          f"neighbor = {neighbor} immediate_parent = {immediate_parent}")
    
    return found, neighbor, immediate_parent

In [11]:
# explode one pair in the tree, as defined by this particular puzzle
def explode(parents, exploding_pair):
    
    verbose(f"  called explode() with parents = {parents} exploding_pair = {exploding_pair}")
    
    left,right = exploding_pair[0], exploding_pair[1]
    
    leftside = find_side_of(exploding_pair,parents,use_strongside = True)
    rightside = find_side_of(exploding_pair,parents,use_strongside = False)
        
    verbose(f"    exploding with leftside = {leftside}")
    verbose(f"    exploding with rightside = {rightside}")
    
    _, neighbor, parent_of_exploding_pair = leftside
    
    if exploding_pair is parent_of_exploding_pair[0]:
        parent_of_exploding_pair[0] = 0
    else:
        parent_of_exploding_pair[1] = 0
    
    if neighbor is not None:
        value, neighbor_parent, on_left = neighbor
        neighbor_index = 0 if on_left else 1
        assert( value == neighbor_parent[ neighbor_index ] )
        neighbor_parent[ neighbor_index ] += left
    
    _, neighbor, _ = rightside
    
    if neighbor is not None:
        value, neighbor_parent, on_left = neighbor
        neighbor_index = 0 if on_left else 1
        assert( value == neighbor_parent[ neighbor_index ] )
        neighbor_parent[ neighbor_index ] += right
        
    return

In [12]:
# split one value in the tree, as defined by this particular puzzle
def split(parents, pair, splitting_value, on_left):
    
    print(f"  called split() with parents = {parents} pair = {pair} splitting_value = {splitting_value}")

    left = splitting_value // 2
    right = splitting_value - left
    new_pair = [left,right]
    
    #found, _, immediate_parent = find_side_of(pair,parents)
    #assert(found)
    
    if on_left:
        pair[0] = new_pair
    else:
        pair[1] = new_pair
        
    print(f"  exiting split() with parents = {parents}")
    
    assert(False)
    
    return

In [13]:
# make one reduction pass through the tree, caller will repeat until no reductions remain
def reduce(pair, parent = None, depth = 0):
    
    print(f"  called reduce() with {pair}, depth = {depth}")
    
    global global_endless_loop
    global_endless_loop -= 1
    assert( global_endless_loop > 0 )
    
    left,right = pair[0],pair[1]
    reduced = False
    
    if parent is None:
        parent = pair
    
    while True:
        pair[0],pair[1] = left,right
    
        if isinstance(left,int) and isinstance(right,int):
            # reg number
            if depth >= 4:
                # explode!
                #print(f"explode {pair}, depth = {depth}")
                explode(parent,pair)
                reduced = True
                
            elif isinstance(left,int) and left >= 10:
                # split left!
                split(parent, pair, left, True)
                reduced = True
            
            elif isinstance(right,int) and right >= 10:
                # split right!
                split(parent, pair, right, False)
                reduced = True
            
        else:
            # inner pairs or mixed regular-and-pair
            if not isinstance(left,int):
                if reduce(left, parent, depth + 1):
                    reduced = True
                    
            if not reduced and not isinstance(right,int):
                if reduce(right, parent, depth + 1):
                    reduced = True
                    
            if not reduced and isinstance(left,int) and left >= 10:
                # split left!
                split(parent, pair, left, True)
                reduced = True
            
            if not reduced and isinstance(right,int) and right >= 10:
                # split right!
                split(parent, pair, right, False)
                reduced = True
            
        break # old: we didn't reduce on this pass through the loop
            # new: we probably don't need this loop at all anymore, with no continue
    
    return reduced

# some test reductions from the original puzzle description
test_reduce = {} # test-input --> expected-result
test_reduce["[[[[0,9],2],3],4]"] = [[[[[9,8],1],2],3],4]
test_reduce["[7,[6,[5,[7,0]]]]"] = [7,[6,[5,[4,[3,2]]]]]
test_reduce["[[6,[5,[7,0]]],3]"] = [[6,[5,[4,[3,2]]]],1]
test_reduce["[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]"] = [[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]
test_reduce["[[3,[2,[8,0]]],[9,[5,[7,0]]]]"] = [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]

for test_expected,test_pair in test_reduce.items():
    print(f"{60 * '='}\nTesting reduce( {test_pair} )")
    while reduce(test_pair):
        pass
    print(f"Resulting reduce() = {test_pair}")
    print(f"Expecting reduce() = {test_expected.replace(',',', ')}")

Testing reduce( [[[[[9, 8], 1], 2], 3], 4] )
  called reduce() with [[[[[9, 8], 1], 2], 3], 4], depth = 0
  called reduce() with [[[[9, 8], 1], 2], 3], depth = 1
  called reduce() with [[[9, 8], 1], 2], depth = 2
  called reduce() with [[9, 8], 1], depth = 3
  called reduce() with [9, 8], depth = 4
  called reduce() with [[[[0, 9], 2], 3], 4], depth = 0
  called reduce() with [[[0, 9], 2], 3], depth = 1
  called reduce() with [[0, 9], 2], depth = 2
  called reduce() with [0, 9], depth = 3
Resulting reduce() = [[[[0, 9], 2], 3], 4]
Expecting reduce() = [[[[0, 9], 2], 3], 4]
Testing reduce( [7, [6, [5, [4, [3, 2]]]]] )
  called reduce() with [7, [6, [5, [4, [3, 2]]]]], depth = 0
  called reduce() with [6, [5, [4, [3, 2]]]], depth = 1
  called reduce() with [5, [4, [3, 2]]], depth = 2
  called reduce() with [4, [3, 2]], depth = 3
  called reduce() with [3, 2], depth = 4
  called reduce() with [7, [6, [5, [7, 0]]]], depth = 0
  called reduce() with [6, [5, [7, 0]]], depth = 1
  called redu

In [14]:
# assert(False) # stop here for now

In [15]:
for source_name in ["demo"]:
    source = data[source_name]
    expected_values = source["expected_values"]
    for i,operands in enumerate(source["operands"]):
        print(f"{source_name}{i} operands({len(operands)}) ")      
        reduced_sum = reduced_sum_of_pairs(operands)
        print(f"  reduced_sum_of_pairs = {reduced_sum}")
        print(f"  expected result: {expected_values[i]}")  

assert(False)

demo0 operands(2) 
  called reduced_sum_of_pairs() with [[[[[4, 3], 4], 4], [7, [[8, 4], 9]]], [1, 1]]
  *** unreduced_sum = [[[[4, 3], 4], 4], [7, [[8, 4], 9]]]
  called reduce() with [[[[4, 3], 4], 4], [7, [[8, 4], 9]]], depth = 0
  called reduce() with [[[4, 3], 4], 4], depth = 1
  called reduce() with [[4, 3], 4], depth = 2
  called reduce() with [4, 3], depth = 3
  called reduce() with [7, [[8, 4], 9]], depth = 1
  called reduce() with [[8, 4], 9], depth = 2
  called reduce() with [8, 4], depth = 3
  *** unreduced_sum = [[[[[4, 3], 4], 4], [7, [[8, 4], 9]]], [1, 1]]
  called reduce() with [[[[[4, 3], 4], 4], [7, [[8, 4], 9]]], [1, 1]], depth = 0
  called reduce() with [[[[4, 3], 4], 4], [7, [[8, 4], 9]]], depth = 1
  called reduce() with [[[4, 3], 4], 4], depth = 2
  called reduce() with [[4, 3], 4], depth = 3
  called reduce() with [4, 3], depth = 4
  called reduce() with [[[[0, 7], 4], [7, [[8, 4], 9]]], [1, 1]], depth = 0
  called reduce() with [[[0, 7], 4], [7, [[8, 4], 9]]], 

AssertionError: 

In [ ]:
for source_name,source in data.items():
    expected_values = source["expected_values"]
    for i,operands in enumerate(source["operands"]):
        print(f"{source_name}{i} operands({len(operands)}):{operands} expect:{expected_values[i]}")
assert(False)

In [ ]:
for source_name in ["demo"]:
    source = data[source_name]
    expected_values = source["expected_values"]
    for i,operands in enumerate(source["operands"]):
        print(f"{source_name}{i} operands({len(operands)}) expect:{expected_values[i]}")        
        unreduced_sum = reduced_sum_of_pairs(operands)
        print(f"  reduced_sum_of_pairs = {unreduced_sum}")

assert(False)

In [ ]:
for source_name,source in data.items():
    expected_values = source["expected_values"]
    for i,operands in enumerate(source["operands"]):
        print(f"{source_name}{i} operands({len(operands)}):{operands} expect:{expected_values[i]}")
assert(False)